In [58]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime, timedelta
import time

In [59]:
# create a function to categorize the time into periods
def categorize_time(time):
    if time.hour < 9 or (time.hour == 9 and time.minute < 30):
        return 'pre-market'
    elif 9 <= time.hour < 16 or (time.hour == 9 and time.minute >= 30):
        return 'intraday'
    else:
        return 'aftermarket'

In [60]:
# Pull in the list of companies

df_SP_500 = pd.read_csv("constituents.csv")
ticker_list = df_SP_500['Symbol'].astype(str).tolist()
ticker_list = [t.replace('.', '-') for t in ticker_list]

for ticker in ticker_list:
    print(ticker)

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BRK-B
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BSX
BMY
AVGO
BR
BRO
BF-B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ

In [62]:
# create the placeholder data frame
holding_df = pd.DataFrame()

for ticker in ticker_list:
    try:
        # Read the CSV file into a DataFrame
        df_test = pd.read_csv(f'Media_Sentiment_Scores\{ticker}_Sentiments.csv')

        # Convert 'time_published' to datetime
        df_test['time_published'] = pd.to_datetime(df_test['time_published'])

        # Add a new column that creates a relevance weighted sentiment score
        df_test['relevance_weighted_sentiment_score'] = df_test['relevance_score'] * df_test['ticker_sentiment_score']

        # Apply the time periods function
        df_test['time_period'] = df_test['time_published'].apply(categorize_time)

        # Define the impacted trading day
        df_test['date'] = df_test['time_published'] + timedelta(hours=8)
        df_test['date'] = df_test['date'].dt.date

        df_test = df_test.drop(['title','url','time_published','summary','ticker','relevance_score','ticker_sentiment_label'], axis=1)

        # # Display the DataFrame
        # display(df_test)

        # Group by 'trading_date' and 'time_period' and calculate mean sentiment scores
        grouped = df_test.groupby(['date', 'time_period']).agg({
                    'ticker_sentiment_score': 'mean',
                    'relevance_weighted_sentiment_score': 'mean'
                    }).reset_index()

        # Pivot the DataFrame
        pivoted = grouped.pivot(index='date', columns='time_period', values=['ticker_sentiment_score', 'relevance_weighted_sentiment_score'])
        pivoted.columns = [f'{col[0]}_{col[1]}' for col in pivoted.columns]
        pivoted.reset_index(inplace=True)
        pivoted.fillna(0, inplace=True)

        # display(pivoted)

        # Read the CSV file into a DataFrame
        df_test_too = pd.read_csv(f'2023_Closing_Stock_Prices\{ticker}_stock_data.csv')

        # Convert 'time_published' to datetime
        df_test_too['date'] = pd.to_datetime(df_test_too['date'])
        pivoted['date'] = pd.to_datetime(pivoted['date'])

        # Perform a left merge
        merged_df = df_test_too.merge(pivoted, on='date', how='left')
        merged_df.reset_index(drop=True, inplace=True)
        merged_df.fillna(0, inplace=True)

        # display(merged_df)

        sector = df_SP_500.loc[df_SP_500["Symbol"] == ticker, "GICS Sector"].values[0]
        merged_df['GICS Sector'] = sector

        industry = df_SP_500.loc[df_SP_500["Symbol"] == ticker, "GICS Sub-Industry"].values[0]
        merged_df['GICS Sub-Industry'] = industry

        # display(merged_df)
        holding_df = pd.concat([holding_df, merged_df], ignore_index=True)

    except:
        print(f"Data not found for {ticker}")

file_path = 'Final_File/combined.csv'
os.makedirs(os.path.dirname(file_path), exist_ok=True)
holding_df.to_csv(file_path, index=False)

print(f"DataFrame exported to {file_path}")
    

Data not found for AOS
Data not found for GOOGL
Data not found for AJG
Data not found for BRK-B
Data not found for BF-B
Data not found for CHRW
Data not found for CE
Data not found for CHD
Data not found for CMCSA
Data not found for CPAY
Data not found for DAY
Data not found for DHI
Data not found for EQR
Data not found for EL
Data not found for FOXA
Data not found for GEV
Data not found for HBAN
Data not found for JBHT
Data not found for JPM
Data not found for KKR
Data not found for LOW
Data not found for MTB
Data not found for MMC
Data not found for MKC
Data not found for MRK
Data not found for MPWR
Data not found for NDAQ
Data not found for NWSA
Data not found for NWS


C:\Users\shawn\AppData\Local\Temp\ipykernel_4892\4252466883.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_test['time_published'] = pd.to_datetime(df_test['time_published'])


Data not found for CRM
Data not found for SJM
Data not found for SW
Data not found for SOLV
Data not found for TROW
Data not found for USB
Data not found for WRB
Data not found for GWW
DataFrame exported to Final_File/combined.csv
